### Generate Spurious-Motif Dataset

In [1]:
from BA3_loc import *
import pickle
from tqdm import tqdm
import os.path as osp
import warnings
warnings.filterwarnings("ignore")

global_b = '0.333' # Set bias degree here
data_dir = f'../data/SPMotif-{global_b}/raw/'
os.makedirs(data_dir, exist_ok=True)

In [2]:
def get_house(basis_type, nb_shapes=80, width_basis=8, feature_generator=None, m=3, draw=True):
    """ Synthetic Graph:

    Start with a tree and attach HOUSE-shaped subgraphs.
    """
    list_shapes = [["house"]] * nb_shapes # house

    if draw:
        plt.figure(figsize=figsize)

    G, role_id, _ = synthetic_structsim.build_graph(
        width_basis, basis_type, list_shapes, start=0, rdm_basis_plugins=True
    )
    G = perturb([G], 0.00, id=role_id)[0]

    if feature_generator is None:
        feature_generator = featgen.ConstFeatureGen(1)
    feature_generator.gen_node_features(G)

    name = basis_type + "_" + str(width_basis) + "_" + str(nb_shapes)

    return G, role_id, name

def get_cycle(basis_type, nb_shapes=80, width_basis=8, feature_generator=None, m=3, draw=True):
    """ Synthetic Graph:

    Start with a tree and attach cycle-shaped (directed edges) subgraphs.
    """
    list_shapes = [["dircycle"]] * nb_shapes

    if draw:
        plt.figure(figsize=figsize)

    G, role_id, _ = synthetic_structsim.build_graph(
        width_basis, basis_type, list_shapes, start=0, rdm_basis_plugins=True
    )
    G = perturb([G], 0.00, id=role_id)[0]       # 0.05 original

    if feature_generator is None:
        feature_generator = featgen.ConstFeatureGen(1)
    feature_generator.gen_node_features(G)

    name = basis_type + "_" + str(width_basis) + "_" + str(nb_shapes)

    return G, role_id, name

def get_crane(basis_type, nb_shapes=80, width_basis=8, feature_generator=None, m=3, draw=True):
    """ Synthetic Graph:

    Start with a tree and attach crane-shaped subgraphs.
    """
    list_shapes = [["varcycle"]] * nb_shapes   # crane

    if draw:
        plt.figure(figsize=figsize)

    G, role_id, _ = synthetic_structsim.build_graph(
        width_basis, basis_type, list_shapes, start=0, rdm_basis_plugins=True
    )
    G = perturb([G], 0.00, id=role_id)[0]

    if feature_generator is None:
        feature_generator = featgen.ConstFeatureGen(1)
    feature_generator.gen_node_features(G)

    name = basis_type + "_" + str(width_basis) + "_" + str(nb_shapes)

    return G, role_id, name

## Training Dataset

In [7]:
edge_index_list, label_list = [], []
ground_truth_list, role_id_list, pos_list = [], [], []
bias = float(global_b)

def graph_stats(base_num):
    if base_num == 1:
        base = 'tree'
        width_basis=np.random.choice(range(3))
    if base_num == 2:
        base = 'ladder'
        width_basis=np.random.choice(range(8,12))
    if base_num == 3:
        base = 'wheel'
        width_basis=np.random.choice(range(15,20))
    return base, width_basis

e_mean, n_mean = [], []
for _ in tqdm(range(1000)):
    base_num = np.random.choice([1,2,3], p=[bias,(1-bias)/2,(1-bias)/2])
    base, width_basis = graph_stats(base_num)

    G, role_id, name = get_cycle(basis_type=base, nb_shapes=1, 
                                    width_basis=width_basis, feature_generator=None, m=3, draw=False)
    label_list.append(0)
    e_mean.append(len(G.edges))
    n_mean.append(len(G.nodes))

    role_id = np.array(role_id)
    edge_index = np.array(G.edges, dtype=np.int32).T

    role_id_list.append(role_id)
    edge_index_list.append(edge_index)
    pos_list.append(np.array(list(nx.spring_layout(G).values())))
    ground_truth_list.append(find_gd(edge_index, role_id))

print("#Graphs: %d    #Nodes: %.2f    #Edges: %.2f " % (len(ground_truth_list), np.mean(n_mean), np.mean(e_mean)))

e_mean, n_mean = [], []
for _ in tqdm(range(1000)):
    base_num = np.random.choice([1,2,3], p=[(1-bias)/2,bias,(1-bias)/2])
    base, width_basis = graph_stats(base_num)

    G, role_id, name = get_house(basis_type=base, nb_shapes=1, 
                                    width_basis=width_basis, feature_generator=None, m=3, draw=False)
    label_list.append(1)
    e_mean.append(len(G.edges))
    n_mean.append(len(G.nodes))

    role_id = np.array(role_id)
    edge_index = np.array(G.edges, dtype=np.int32).T

    role_id_list.append(role_id)
    edge_index_list.append(edge_index)
    pos_list.append(np.array(list(nx.spring_layout(G).values())))
    ground_truth_list.append(find_gd(edge_index, role_id))

print("#Graphs: %d    #Nodes: %.2f    #Edges: %.2f " % (len(ground_truth_list), np.mean(n_mean), np.mean(e_mean)))


e_mean, n_mean = [], []
for _ in tqdm(range(1000)):
    base_num = np.random.choice([1,2,3], p=[(1-bias)/2,(1-bias)/2,bias])
    base, width_basis = graph_stats(base_num)
    
    G, role_id, name = get_crane(basis_type=base, nb_shapes=1, 
                                    width_basis=width_basis, feature_generator=None, m=3, draw=False)
    label_list.append(2)
    e_mean.append(len(G.edges))
    n_mean.append(len(G.nodes))

    role_id = np.array(role_id)
    edge_index = np.array(G.edges, dtype=np.int32).T

    role_id_list.append(role_id)
    edge_index_list.append(edge_index)
    pos_list.append(np.array(list(nx.spring_layout(G).values())))
    ground_truth_list.append(find_gd(edge_index, role_id))

print("#Graphs: %d    #Nodes: %.2f    #Edges: %.2f " % (len(ground_truth_list), np.mean(n_mean), np.mean(e_mean)))

with open(osp.join(data_dir, 'train.npy'), 'wb') as f:
    pickle.dump((edge_index_list, label_list, ground_truth_list, role_id_list, pos_list), f, protocol=pickle.HIGHEST_PROTOCOL)

  0%|          | 0/1000 [00:00<?, ?it/s]

100%|██████████| 1000/1000 [00:03<00:00, 254.95it/s]


#Graphs: 1000    #Nodes: 18.85    #Edges: 27.05 


100%|██████████| 1000/1000 [00:03<00:00, 253.08it/s]


#Graphs: 2000    #Nodes: 18.68    #Edges: 27.97 


100%|██████████| 1000/1000 [00:03<00:00, 256.35it/s]

#Graphs: 3000    #Nodes: 18.82    #Edges: 29.13 


## Val Dataset

In [9]:
edge_index_list, label_list = [], []
ground_truth_list, role_id_list, pos_list = [], [], []
bias = float(global_b)

def graph_stats(base_num):
    if base_num == 1:
        base = 'tree'
        width_basis=np.random.choice(range(3))
    if base_num == 2:
        base = 'ladder'
        width_basis=np.random.choice(range(8,12))
    if base_num == 3:
        base = 'wheel'
        width_basis=np.random.choice(range(15,20))
    return base, width_basis

e_mean, n_mean = [], []
for _ in tqdm(range(1000)):
    base_num = np.random.choice([1,2,3])
    base, width_basis = graph_stats(base_num)

    G, role_id, name = get_cycle(basis_type=base, nb_shapes=1, 
                                    width_basis=width_basis, feature_generator=None, m=3, draw=False)
    label_list.append(0)
    e_mean.append(len(G.edges))
    n_mean.append(len(G.nodes))

    role_id = np.array(role_id)
    edge_index = np.array(G.edges, dtype=np.int32).T

    role_id_list.append(role_id)
    edge_index_list.append(edge_index)
    pos_list.append(np.array(list(nx.spring_layout(G).values())))
    ground_truth_list.append(find_gd(edge_index, role_id))

print("#Graphs: %d    #Nodes: %.2f    #Edges: %.2f " % (len(ground_truth_list), np.mean(n_mean), np.mean(e_mean)))

e_mean, n_mean = [], []
for _ in tqdm(range(1000)):
    base_num = np.random.choice([1,2,3])
    base, width_basis = graph_stats(base_num)

    G, role_id, name = get_house(basis_type=base, nb_shapes=1, 
                                    width_basis=width_basis, feature_generator=None, m=3, draw=False)
    label_list.append(1)
    e_mean.append(len(G.edges))
    n_mean.append(len(G.nodes))

    role_id = np.array(role_id)
    edge_index = np.array(G.edges, dtype=np.int32).T

    role_id_list.append(role_id)
    edge_index_list.append(edge_index)
    pos_list.append(np.array(list(nx.spring_layout(G).values())))
    ground_truth_list.append(find_gd(edge_index, role_id))

print("#Graphs: %d    #Nodes: %.2f    #Edges: %.2f " % (len(ground_truth_list), np.mean(n_mean), np.mean(e_mean)))


e_mean, n_mean = [], []
for _ in tqdm(range(1000)):
    base_num = np.random.choice([1,2,3])
    base, width_basis = graph_stats(base_num)
    
    G, role_id, name = get_crane(basis_type=base, nb_shapes=1, 
                                    width_basis=width_basis, feature_generator=None, m=3, draw=False)
    label_list.append(2)
    e_mean.append(len(G.edges))
    n_mean.append(len(G.nodes))

    role_id = np.array(role_id)
    edge_index = np.array(G.edges, dtype=np.int32).T

    role_id_list.append(role_id)
    edge_index_list.append(edge_index)
    pos_list.append(np.array(list(nx.spring_layout(G).values())))
    ground_truth_list.append(find_gd(edge_index, role_id))

print("# Graphs: %d    # Nodes: %.2f    # Edges: %.2f " % (len(ground_truth_list), np.mean(n_mean), np.mean(e_mean)))

with open(osp.join(data_dir, 'val.npy'), 'wb') as f:
    pickle.dump((edge_index_list, label_list, ground_truth_list, role_id_list, pos_list), f, protocol=pickle.HIGHEST_PROTOCOL)

  0%|          | 0/1000 [00:00<?, ?it/s]

100%|██████████| 1000/1000 [00:03<00:00, 259.40it/s]


#Graphs: 1000    #Nodes: 18.11    #Edges: 26.18 


100%|██████████| 1000/1000 [00:03<00:00, 258.45it/s]


#Graphs: 2000    #Nodes: 18.52    #Edges: 27.75 


100%|██████████| 1000/1000 [00:03<00:00, 255.18it/s]


# Graphs: 3000    # Nodes: 19.01    # Edges: 29.48 


## Testing Dataset

In [10]:
# no bias for test dataset
edge_index_list, label_list = [], []
ground_truth_list, role_id_list, pos_list = [], [], []

def graph_stats_large(base_num):
    if base_num == 1:
        base = 'tree'
        width_basis=np.random.choice(range(3,6))
    if base_num == 2:
        base = 'ladder'
        width_basis=np.random.choice(range(30,50))
    if base_num == 3:
        base = 'wheel'
        width_basis=np.random.choice(range(60,80))
    return base, width_basis

e_mean, n_mean = [], []
for _ in tqdm(range(2000)):
    base_num = np.random.choice([1,2,3]) # uniform
    base, width_basis = graph_stats_large(base_num)

    G, role_id, name = get_cycle(basis_type=base, nb_shapes=1, 
                                    width_basis=width_basis, feature_generator=None, m=3, draw=False)
    label_list.append(0)
    e_mean.append(len(G.edges))
    n_mean.append(len(G.nodes))

    role_id = np.array(role_id)
    edge_index = np.array(G.edges, dtype=np.int32).T

    role_id_list.append(role_id)
    edge_index_list.append(edge_index)
    pos_list.append(np.array(list(nx.spring_layout(G).values())))
    ground_truth_list.append(find_gd(edge_index, role_id))

print("#Graphs: %d    #Nodes: %.2f    #Edges: %.2f " % (len(ground_truth_list), np.mean(n_mean), np.mean(e_mean)))

e_mean, n_mean = [], []
for _ in tqdm(range(2000)):
    base_num = np.random.choice([1,2,3])
    base, width_basis = graph_stats_large(base_num)

    G, role_id, name = get_house(basis_type=base, nb_shapes=1, 
                                    width_basis=width_basis, feature_generator=None, m=3, draw=False)
    label_list.append(1)
    e_mean.append(len(G.edges))
    n_mean.append(len(G.nodes))

    role_id = np.array(role_id)
    edge_index = np.array(G.edges, dtype=np.int32).T

    role_id_list.append(role_id)
    edge_index_list.append(edge_index)
    pos_list.append(np.array(list(nx.spring_layout(G).values())))
    ground_truth_list.append(find_gd(edge_index, role_id))

print("#Graphs: %d    #Nodes: %.2f    #Edges: %.2f " % (len(ground_truth_list), np.mean(n_mean), np.mean(e_mean)))

e_mean, n_mean = [], []
for _ in tqdm(range(2000)):
    base_num = np.random.choice([1,2,3])
    base, width_basis = graph_stats_large(base_num)

    G, role_id, name = get_crane(basis_type=base, nb_shapes=1, 
                                    width_basis=width_basis, feature_generator=None, m=3, draw=False)
    label_list.append(2)
    e_mean.append(len(G.edges))
    n_mean.append(len(G.nodes))

    role_id = np.array(role_id)
    edge_index = np.array(G.edges, dtype=np.int32).T

    role_id_list.append(role_id)
    edge_index_list.append(edge_index)
    pos_list.append(np.array(list(nx.spring_layout(G).values())))
    ground_truth_list.append(find_gd(edge_index, role_id))

print("#Graphs: %d    #Nodes: %.2f    #Edges: %.2f " % (len(ground_truth_list), np.mean(n_mean), np.mean(e_mean)))
with open(osp.join(data_dir, 'test.npy'), 'wb') as f:
    pickle.dump((edge_index_list, label_list, ground_truth_list, role_id_list, pos_list), f, protocol=pickle.HIGHEST_PROTOCOL)

100%|██████████| 2000/2000 [00:48<00:00, 41.65it/s]


#Graphs: 2000    #Nodes: 89.75    #Edges: 125.26 


100%|██████████| 2000/2000 [00:47<00:00, 42.16it/s]


#Graphs: 4000    #Nodes: 89.99    #Edges: 126.62 


100%|██████████| 2000/2000 [00:46<00:00, 42.73it/s]


#Graphs: 6000    #Nodes: 89.54    #Edges: 127.04 
